# LSTM Analysis

In [11]:
import h5py
import pandas as pd
import numpy as np

## Read dataset

In [22]:
dataset_file = "users.h5"
users_df = pd.read_hdf(dataset_file)
users_df

bs,0,1,2,3,10,11,12,13,20,21,...,972,973,980,982,983,990,991,992,993,99300
time_hour,,,,,,,,,,,,,,,,,,,,,
2012-08-18 16:00:00+00:00,103,32,35,36,1919,667,610,642,1694,572,...,20,47,32,11,21,609,208,194,207,161282
2012-08-18 17:00:00+00:00,78,31,20,27,1348,462,429,457,1226,416,...,15,37,16,7,9,377,136,126,115,108600
2012-08-18 18:00:00+00:00,36,17,9,10,914,304,286,324,929,315,...,10,28,14,5,9,212,88,70,54,77076
2012-08-18 19:00:00+00:00,32,14,7,11,653,210,208,235,799,266,...,8,22,11,4,7,160,65,44,51,60118
2012-08-18 20:00:00+00:00,23,7,5,11,551,183,174,194,834,258,...,15,21,8,2,6,153,58,42,53,54592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-08-26 11:00:00+00:00,235,84,94,57,846,357,188,301,3308,1064,...,55,90,38,11,27,939,260,315,364,244852
2012-08-26 12:00:00+00:00,219,81,77,61,928,381,207,340,3314,1058,...,64,88,41,16,25,941,278,313,350,241728
2012-08-26 13:00:00+00:00,187,67,63,57,771,303,172,296,2966,1021,...,50,79,36,17,19,876,262,299,315,220012


### LSTM Config

In [26]:
learning_rate = 0.01
epochs = 7000
batch_size = 64
num_input = users_df.shape[1]
timesteps = users_df.shape[0]
keep_rate_DROPOUT = 1;
optimizer = "adam"
loss_function = "mean_squared_root"
lstm_units = 50


In [27]:
print("Learning rate:                   ", learning_rate)
print("Training steps:                  ", training_step)
print("Keep Rate DROPOUT:               ", keep_rate_DROPOUT)

print("Number of input(Metrics):        ", num_input)
print("Time steps:                      ", timesteps)
print("Batch size:                      ", batch_size)


Learning rate:                    0.01
Training steps:                   7000
Keep Rate DROPOUT:                1
Number of input(Metrics):         400
Time steps:                       192
Batch size:                       64
